In [1]:
import pandas as pd
import numpy as np
import pymongo                                                  
from dotenv import load_dotenv
import os
import time
from datetime import datetime
import nba_api_functions_hist as hist
import nba_prop_functions_hist as prop_hist
import random
import uuid

In [2]:
def fetch_data_from_database():
    # connecting to mongodb
    load_dotenv()
    uri = os.getenv('URI')
    client = pymongo.MongoClient(uri)
    db = client['nba_stats']
    
    # creating pandas df for historical players logs
    hist_player_stats = db['hist_player_logs']
    player_cursor = hist_player_stats.find()
    hist_player_stats_df = pd.DataFrame(list(player_cursor))
    hist_player_stats_df = hist_player_stats_df.drop(columns='_id')

    return hist_player_stats_df

hist_df = fetch_data_from_database()

In [3]:
hist_df

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22022,1630173,22201221,"APR 09, 2023",TOR vs. MIL,W,28,6,11,0.545,...,12,13,1,2,0,2,2,14,9,1
1,22022,1630173,22201206,"APR 07, 2023",TOR @ BOS,L,24,6,10,0.600,...,5,7,0,1,0,0,3,16,-9,1
2,22022,1630173,22201068,"MAR 19, 2023",TOR @ MIL,L,20,2,3,0.667,...,4,5,3,0,1,0,1,5,7,1
3,22022,1630173,22201029,"MAR 14, 2023",TOR vs. DEN,W,14,0,3,0.000,...,1,4,1,0,0,0,0,4,-13,1
4,22022,1630173,22201003,"MAR 10, 2023",TOR @ LAL,L,8,2,5,0.400,...,1,2,0,0,0,1,2,5,-10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101184,22016,1627826,21600367,"DEC 12, 2016",LAL @ SAC,L,2,0,1,0.000,...,0,0,0,0,0,1,0,0,-5,1
101185,22016,1627826,21600326,"DEC 07, 2016",LAL @ HOU,L,5,0,2,0.000,...,0,0,0,0,0,0,0,2,-1,1
101186,22016,1627826,21600142,"NOV 13, 2016",LAL @ MIN,L,8,0,2,0.000,...,2,2,0,0,0,1,2,1,-4,1
101187,22016,1627826,21600078,"NOV 04, 2016",LAL vs. GSW,W,1,0,0,0.000,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# filtering out only what we need and changing the names to match
hist_df = hist_df[['SEASON_ID','Game_ID','GAME_DATE','Player_ID','MIN','PTS','REB','AST','STL','BLK','FG3M']]
hist_df.rename(columns={'SEASON_ID':'season_id','Game_ID':'game_id','GAME_DATE':'game_date','Player_ID':'player_id', 'MIN':'min', 
                        'PTS':'pts','REB':'reb','AST':'ast','STL':'stl','BLK':'blk', 'FG3M':'fg3m'}, inplace=True)

# updating game_date to something sortable
# ******* not sure if we need to flip it back at somepoint
hist_df['game_date'] = pd.to_datetime(hist_df['game_date'], format='%b %d, %Y')
# # ***** returns it back to a more readable format ********
# hist_df['game_date'] = hist_df['game_date'].dt.strftime('%b %d, %Y')

# sorting by playerid, seasonid, gamedate
hist_df = hist_df.sort_values(by=['player_id', 'season_id', 'game_date'], ascending=True).reset_index(drop=True)

# grouping by playerid and seasonid to add player_game_no
hist_df['player_game_no'] = hist_df.groupby(['season_id', 'player_id']).cumcount() + 1

# adding unique id to each row
hist_df['unique_id'] = hist_df.index.map(lambda _:uuid.uuid4())

In [ ]:
# ******** NEED TO UPDATE THIS TO A ROLLING IN SEASON AVERAGE ********
hist_df['min_season_avg'] = hist_df.groupby(['season_id', 'player_id'])['min'].transform('mean')
hist_df['pts_season_avg'] = hist_df.groupby(['season_id', 'player_id'])['pts'].transform('mean')
hist_df['reb_season_avg'] = hist_df.groupby(['season_id', 'player_id'])['reb'].transform('mean')
hist_df['ast_season_avg'] = hist_df.groupby(['season_id', 'player_id'])['ast'].transform('mean')
hist_df['stl_season_avg'] = hist_df.groupby(['season_id', 'player_id'])['stl'].transform('mean')
hist_df['blk_season_avg'] = hist_df.groupby(['season_id', 'player_id'])['blk'].transform('mean')
hist_df['fg3m_season_avg'] = hist_df.groupby(['season_id', 'player_id'])['fg3m'].transform('mean')

In [ ]:
memory_usage = pts_results_df.memory_usage(deep=True).sum()
memory_usage = memory_usage / (1024**2)
memory_usage